<a href="https://colab.research.google.com/github/casllmproject/wjs_llm_project/blob/main/ai_shoes_gpt_4o_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: mount goolge drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install openai
!pip install openai==0.28
!pip install openai python-dotenv

In [ ]:
import pandas as pd
import openai
import numpy as np
import json
import time
import os
import re
import random
from tqdm import tqdm

In [ ]:
openai.api_key = open("/content/drive/MyDrive/CASLLM/casllm_apikey.txt", "r").read().strip("\n")

In [ ]:
# Load the survey questions from the CSV file
survey_df = pd.read_csv('/content/drive/MyDrive/AI_shoes/survey_questions.csv')

In [ ]:
def gpt_take_survey(model_name, temperature=0.7, max_retries=3):
    responses = []

    # Extract the context from the CSV file
    context = survey_df[survey_df['Variable_Name'] == 'context']['Question'].values[0]

    # Track time
    start_time = time.time()

    # Loop through each survey question with a progress bar
    for i, row in tqdm(enumerate(survey_df.iterrows()), total=len(survey_df), desc=f'Processing {model_name}'):
        if row[1]['Variable_Name'] == 'context':
            continue  # Skip the context row

        # Prepare the prompt
        prompt = f"{context}\n\n{row[1]['Question']}\nOptions: {row[1]['Options']}\nChoose one of the options."

        # Implement retry mechanism
        for attempt in range(max_retries):
            try:
                # Make the API call using ChatCompletion
                response = openai.ChatCompletion.create(
                    model=model_name,
                    messages=[
                        {"role": "system", "content": context},
                        {"role": "user", "content": prompt}
                    ],
                    temperature=temperature
                )

                # Extract the model's response
                answer = response['choices'][0]['message']['content'].strip()

                # Extract the number from the response using regular expression
                match = re.search(r'\b[1-5]\b', answer)
                if match:
                    answer = match.group(0)
                else:
                    answer = None  # Handle cases where the model's response doesn't match any option

                # Store the response
                responses.append({
                    "Model": model_name,
                    "Variable_Name": row[1]['Variable_Name'],
                    "Question": row[1]['Question'],
                    "Response": answer
                })
                break  # If successful, break out of retry loop

            except openai.error.APIError as e:
                print(f"Attempt {attempt + 1} failed with error: {e}")
                time.sleep(random.uniform(1, 3))  # Wait a bit before retrying

        # Update progress and time estimate
        elapsed_time = time.time() - start_time
        questions_remaining = len(survey_df) - i - 1
        time_per_question = elapsed_time / (i + 1)
        time_remaining = questions_remaining * time_per_question
        tqdm.write(f"Estimated time remaining: {time.strftime('%H:%M:%S', time.gmtime(time_remaining))}")

    # Convert the responses to a DataFrame and save to CSV in the specified folder
    gpt_df = pd.DataFrame(responses)
    output_path = f'/content/drive/MyDrive/AI_shoes/{model_name}_responses.csv'
    gpt_df.to_csv(output_path, index=False)

    print(f"Responses from {model_name} saved to {output_path}.")

# Example: Run the survey with GPT-4o-mini, GPT-4o
models = ['gpt-4o-2024-08-06']
for model in models:
    gpt_take_survey(model)

In [ ]:
gpt4o_flag = pd.read_csv("/content/drive/MyDrive/AI_shoes/gpt-4o-2024-08-06_responses.csv", encoding='utf-8')
print(gpt4o_flag)

                Model Variable_Name  \
0   gpt-4o-2024-08-06        role_A   
1   gpt-4o-2024-08-06        role_B   
2   gpt-4o-2024-08-06        role_C   
3   gpt-4o-2024-08-06        role_D   
4   gpt-4o-2024-08-06        role_E   
5   gpt-4o-2024-08-06        role_F   
6   gpt-4o-2024-08-06        role_G   
7   gpt-4o-2024-08-06        role_H   
8   gpt-4o-2024-08-06        role_I   
9   gpt-4o-2024-08-06        role_J   
10  gpt-4o-2024-08-06        role_K   
11  gpt-4o-2024-08-06        role_L   
12  gpt-4o-2024-08-06        role_M   
13  gpt-4o-2024-08-06        role_N   
14  gpt-4o-2024-08-06        role_O   
15  gpt-4o-2024-08-06        role_P   
16  gpt-4o-2024-08-06        role_Q   
17  gpt-4o-2024-08-06        role_R   
18  gpt-4o-2024-08-06        role_S   
19  gpt-4o-2024-08-06        role_T   
20  gpt-4o-2024-08-06        role_U   
21  gpt-4o-2024-08-06        role_X   
22  gpt-4o-2024-08-06        role_Y   
23  gpt-4o-2024-08-06        role_V   
24  gpt-4o-2024-08-06    